In [1]:
# Import Dependencies
import pandas as pd

# Create reference to CSV file
fpath = "Resources/selfscoutdata.xlsx"

#pd.reset_option("all")
#pd.set_option('precision', 0)

# Import the CSV into a pandas DataFrame
raw = pd.read_excel(fpath)
raw["Name"]= raw["Name"].str.split(",", n = 1, expand = True)

In [2]:
orp = raw[["FORM","Down","Distance","R/P",]]
#orp = orp.loc[(orp["Down"] == 2) & (orp["Distance"] < 11)]
#orp = orp.loc[(orp["Down"] == 2)]
#orp
#orp["Distance"].describe()

In [3]:
cut_labels_5 = ['1', '2-3', '4-6', '7-10', '11+']
cut_bins = [0, 1, 3, 6, 10, 49]

#pd.cut(orp['Distance'], bins=cut_bins, labels=cut_labels_5)

orp['YD'] = pd.cut(orp['Distance'], bins=cut_bins, labels=cut_labels_5)
orp = orp[["FORM","Down","R/P","YD"]]
#orp

<ipython-input-3-b9ab2a1c8e52>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orp['YD'] = pd.cut(orp['Distance'], bins=cut_bins, labels=cut_labels_5)


In [4]:
run = orp.loc[orp["R/P"] == "R"]

run = run.groupby(["Down","YD"])["R/P"].count().reset_index(name="R")
#run = run.groupby(["Down","YD","FORM"])["R"].count().reset_index()
#run

In [5]:
pas = orp.loc[orp["R/P"] == "P"]

pas = pas.groupby(["Down","YD"])["R/P"].count().reset_index(name="P")

In [6]:
mer = run.merge(pas, on=["Down","YD"])
#mer

In [118]:
#### THIS IS A KEEPER ####
fin = mer.set_index(["Down","YD"])
tt = (fin.div(fin.sum(axis=1), axis=0)*100).round(0).astype(str) + '%'
mer1 = fin.merge(tt, on=["Down","YD"])
mer1.rename(columns={'R_x': 'R', 'P_x': 'P', 'R_y': 'R%', 'P_y': 'P%'}, inplace=True)
mer1 = mer1.reset_index()
#mer1
tdh = mer1.set_index(["Down","YD"])[['R','P','R%','P%']].T.apply(tuple).to_dict('dict')
tdh[1, '1']['R%']

'50.0%'

In [8]:
te = orp.loc[orp["R/P"] == "R"]

te = te.groupby(["Down","YD","FORM"])["R/P"].count().reset_index(name="R")
te = te[te['R'] > 0]
#te.T.to_dict('list')

In [9]:
tp = orp.loc[orp["R/P"] == "P"]

tp = tp.groupby(["Down","YD","FORM"])["R/P"].count().reset_index(name="P")
tp = tp[tp['P'] > 0]
#tp

In [74]:
mr = te.merge(tp, on=["Down","YD","FORM"], how='outer')
mr["R"].fillna(0,inplace=True)
mr["P"].fillna(0,inplace=True)

In [73]:
fn = mr.set_index(["Down","YD","FORM"])
tt1 = (fn.div(fn.sum(axis=1), axis=0)*100).round(0).astype(str) + '%'
merf = fn.merge(tt1, on=["Down","YD","FORM"])
merf.rename(columns={'R_x': 'R', 'P_x': 'P', 'R_y': 'R%', 'P_y': 'P%'}, inplace=True)
merf = merf.reset_index()

In [85]:
mydict = merf.set_index(["Down","YD","FORM"])[['R','P','R%','P%']].T.apply(tuple).to_dict('dict')

In [70]:
def partial_match(key, d):
    for k, v in d.items():
        if all(k1 == k2 or k2 is None for k1, k2 in zip(k, key)):
            yield k[2],v

In [125]:
x = list(partial_match((3, None, None), mydict))
cats = ['1', '2-3', '4-6', '7-10', '11+']
newd = {}

for a in range(1,5):
    for cat in cats:
        newd[str(a)+"_"+cat] = list(partial_match((a, cat, None), mydict))
e = newd["1_1"]
e[0]

('TURBO', {'R': 1.0, 'P': 0.0, 'R%': '100.0%', 'P%': '0.0%'})